# 요기요 리뷰 음식 이미지 크롤링
### 음식당 한 업체 선정해서 100장씩
- 돈까스, 샌드위치, 초밥
- 돈까스 : 저스트카츠
- 샌드위치 : 죠스샌드위치
- 초밥 : 김뿌라스시

In [1]:
import os
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

# 한글대응
from matplotlib import rc
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
rc('font',family='Malgun Gothic')

### 돈까스 : 마당돈까스


In [2]:
url = 'https://www.yogiyo.co.kr/mobile/#/265032/'
driver = webdriver.Chrome('../../driver/118/chromedriver.exe')
driver.get(url)

In [5]:
loc_btn = driver.find_element(By.CSS_SELECTOR, '#search > div > form > input')
loc_btn.clear()
loc_btn.click()
loc_btn.send_keys('서울특별시 서초구 서초동 1498-5 마제스타시티, 힐스테이트 서리풀 힐스테이트 서리풀 102동')
search_btn = driver.find_element(By.CSS_SELECTOR, '#button_search_address > button.btn.btn-default.ico-pick')
search_btn.click()

In [61]:
def get_menu_list(url, addr, rest, menu, review_list, order_list, image_list):
    url = url
    driver = webdriver.Chrome('../../driver/118/chromedriver.exe')
    driver.get(url)

    time.sleep(0.5)

    # 전체화면으로 전환
    driver.maximize_window()

    time.sleep(0.5)

    # 요기요 주소 입력
    loc_btn = driver.find_element(By.CSS_SELECTOR, '#search > div > form > input')
    loc_btn.clear()
    loc_btn.click()
    loc_btn.send_keys(addr)
    search_btn = driver.find_element(By.CSS_SELECTOR, '#button_search_address > button.btn.btn-default.ico-pick')
    search_btn.click()

    time.sleep(1)

    # 전체보기에서 음식점 검색
    search_rest = driver.find_element(By.CSS_SELECTOR, '#category > ul > li.hidden-xs.menu-search > a')
    search_rest.click()
    search_input = driver.find_element(By.CSS_SELECTOR,' #category > ul > li.main-search > form > div > input')
    search_input.send_keys(rest)
    search_input_btn = driver.find_element(By.CSS_SELECTOR, '#category_search_button')
    search_input_btn.click()

    time.sleep(2)

    # 음식점
    rest_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[5]/div/div/div')
    rest_btn.click()


    # 클린리뷰로 이동
    clean_review_btn = driver.find_element(By.CSS_SELECTOR,'#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > ul > li:nth-child(2) > a')
    clean_review_btn.click()

    time.sleep(0.5)

    # 사진리뷰만 보기
    only_img_btn = driver.find_element(By.CSS_SELECTOR, '#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > div:nth-child(6) > div.review-count > div > div > div > label')
    only_img_btn.click()

    time.sleep(0.5)

    # 더보기
    # 최초 10개 이후 더보기마다 10개씩 추가
    for i in range(9):

        # 스크롤 맨 아래로
        down_tag = driver.find_element(By.CSS_SELECTOR, '#review > li.list-group-item.btn-more > a')
        action = ActionChains(driver)
        action.move_to_element(down_tag).perform()

        # 더보기 클릭
        down_tag.click()
        time.sleep(1)

    # 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    reviews = soup.select('#review')
    contents = reviews[0].find_all('li', class_='list-group-item star-point ng-scope')

    # 모든 리뷰들을 돌면서
    idx = 1
    for content in contents:
        
        # 리뷰작성자
        reviewer = content.find('span', class_='review-id ng-binding').text[:-1]
        
        # 주문 메뉴
        order_menu = content.find('div', class_='order-items default ng-binding').text.split('\n')[1]
        order_menu = order_menu.replace(' ','')
        
        # 리뷰 이미지
        image_url = content.find_all('img', class_='ng-isolate-scope')
        
        for cnt in range(len(image_url)):

            image_title = f'{menu}{idx}'
            image_list.append(image_title)
            reviewer_list.append(reviewer)
            order_list.append(order_menu)

            idx += 1

    # 모든 이미지 저장
    os.mkdir(f'./{menu}')

    images = reviews[0].select('table>tbody>tr>td>div>img')

    image_urls = [each['data-url'] for each in images]

    for idx,url in enumerate(image_urls):
        response = requests.get(url)

        if response.status_code == 200:
            image_file = os.path.join(f'./{menu}/',f'{menu}{idx+1}.jpg')
            with open(image_file, 'wb') as file:
                file.write(response.content)
            print('이미지 저장', image_file)
        else:
            print('이미지 저장 실패')    

In [62]:
url = 'https://www.yogiyo.co.kr/mobile/#/'
addr = '서울특별시 서초구 서초동 1498-5 마제스타시티, 힐스테이트 서리풀 힐스테이트 서리풀 102동'
reviewer_list = []
menu_list = []
image_list = []
get_menu_list(url, addr, '저스트카츠', '돈까스', reviewer_list, menu_list, image_list)

이미지 저장 ./돈까스/돈까스1.jpg
이미지 저장 ./돈까스/돈까스2.jpg
이미지 저장 ./돈까스/돈까스3.jpg
이미지 저장 ./돈까스/돈까스4.jpg
이미지 저장 ./돈까스/돈까스5.jpg
이미지 저장 ./돈까스/돈까스6.jpg
이미지 저장 ./돈까스/돈까스7.jpg
이미지 저장 ./돈까스/돈까스8.jpg
이미지 저장 ./돈까스/돈까스9.jpg
이미지 저장 ./돈까스/돈까스10.jpg
이미지 저장 ./돈까스/돈까스11.jpg
이미지 저장 ./돈까스/돈까스12.jpg
이미지 저장 ./돈까스/돈까스13.jpg
이미지 저장 ./돈까스/돈까스14.jpg
이미지 저장 ./돈까스/돈까스15.jpg
이미지 저장 ./돈까스/돈까스16.jpg
이미지 저장 ./돈까스/돈까스17.jpg
이미지 저장 ./돈까스/돈까스18.jpg
이미지 저장 ./돈까스/돈까스19.jpg
이미지 저장 ./돈까스/돈까스20.jpg
이미지 저장 ./돈까스/돈까스21.jpg
이미지 저장 ./돈까스/돈까스22.jpg
이미지 저장 ./돈까스/돈까스23.jpg
이미지 저장 ./돈까스/돈까스24.jpg
이미지 저장 ./돈까스/돈까스25.jpg
이미지 저장 ./돈까스/돈까스26.jpg
이미지 저장 ./돈까스/돈까스27.jpg
이미지 저장 ./돈까스/돈까스28.jpg
이미지 저장 ./돈까스/돈까스29.jpg
이미지 저장 ./돈까스/돈까스30.jpg
이미지 저장 ./돈까스/돈까스31.jpg
이미지 저장 ./돈까스/돈까스32.jpg
이미지 저장 ./돈까스/돈까스33.jpg
이미지 저장 ./돈까스/돈까스34.jpg
이미지 저장 ./돈까스/돈까스35.jpg
이미지 저장 ./돈까스/돈까스36.jpg
이미지 저장 ./돈까스/돈까스37.jpg
이미지 저장 ./돈까스/돈까스38.jpg
이미지 저장 ./돈까스/돈까스39.jpg
이미지 저장 ./돈까스/돈까스40.jpg
이미지 저장 ./돈까스/돈까스41.jpg
이미지 저장 ./돈까스/돈까스42.jpg
이미지 저장 ./돈까스/돈까스43.jpg
이미지 저장 ./돈까스/돈까스44.j

In [63]:
len(reviewer_list), len(menu_list), len(image_list)

(106, 106, 106)

In [64]:
image_list

['돈까스1',
 '돈까스2',
 '돈까스3',
 '돈까스4',
 '돈까스5',
 '돈까스6',
 '돈까스7',
 '돈까스8',
 '돈까스9',
 '돈까스10',
 '돈까스11',
 '돈까스12',
 '돈까스13',
 '돈까스14',
 '돈까스15',
 '돈까스16',
 '돈까스17',
 '돈까스18',
 '돈까스19',
 '돈까스20',
 '돈까스21',
 '돈까스22',
 '돈까스23',
 '돈까스24',
 '돈까스25',
 '돈까스26',
 '돈까스27',
 '돈까스28',
 '돈까스29',
 '돈까스30',
 '돈까스31',
 '돈까스32',
 '돈까스33',
 '돈까스34',
 '돈까스35',
 '돈까스36',
 '돈까스37',
 '돈까스38',
 '돈까스39',
 '돈까스40',
 '돈까스41',
 '돈까스42',
 '돈까스43',
 '돈까스44',
 '돈까스45',
 '돈까스46',
 '돈까스47',
 '돈까스48',
 '돈까스49',
 '돈까스50',
 '돈까스51',
 '돈까스52',
 '돈까스53',
 '돈까스54',
 '돈까스55',
 '돈까스56',
 '돈까스57',
 '돈까스58',
 '돈까스59',
 '돈까스60',
 '돈까스61',
 '돈까스62',
 '돈까스63',
 '돈까스64',
 '돈까스65',
 '돈까스66',
 '돈까스67',
 '돈까스68',
 '돈까스69',
 '돈까스70',
 '돈까스71',
 '돈까스72',
 '돈까스73',
 '돈까스74',
 '돈까스75',
 '돈까스76',
 '돈까스77',
 '돈까스78',
 '돈까스79',
 '돈까스80',
 '돈까스81',
 '돈까스82',
 '돈까스83',
 '돈까스84',
 '돈까스85',
 '돈까스86',
 '돈까스87',
 '돈까스88',
 '돈까스89',
 '돈까스90',
 '돈까스91',
 '돈까스92',
 '돈까스93',
 '돈까스94',
 '돈까스95',
 '돈까스96',
 '돈까스97',
 '돈까스98',
 '돈까스99',
 '돈까스100',
 '돈까스101

In [66]:
pork_cutlet_df = pd.DataFrame({'reviewer':reviewer_list, 'menu':menu_list, 'image':image_list})
pork_cutlet_df.head(20)

,reviewer,menu,image
0,al**,저스트등심돈카츠도시락/1(리뷰이벤트입니다:）(펩시245ml)),돈까스1
1,wo**,"저스트냉모밀세트/1(도시락돈카츠선택1(생안심돈카츠1p),도시락돈카츠선택2(통체다치즈...",돈까스2
2,ko**,"저스트커스텀돈카츠도시락/1(도시락돈카츠선택1(생등심돈카츠1p),도시락돈카츠선택2(생...",돈까스3
3,0r**,"생안심돈까스/1,통치즈롤돈까스/1,생등심돈까스/1,저스트카레우동/1",돈까스4
4,ai**,"저스트커스텀돈카츠도시락/1(도시락돈카츠선택1(생등심돈카츠1p),도시락돈카츠선택2(통...",돈까스5
5,ym**,"저스트등심돈카츠도시락/1(돈카츠추가선택(생안심돈카츠추가),추가선택(우동장국추가),사...",돈까스6
6,na**,"저스트등심돈카츠도시락/1(사이드메뉴추가선택(감자고로케4개추가),리뷰이벤트입니다:）(...",돈까스7
7,do**,"저스트냉모밀세트/1(도시락돈카츠선택1(생등심돈카츠1p),도시락돈카츠선택2(통치즈돈카...",돈까스8
8,ha**,"저스트안심돈카츠도시락/1(사이드메뉴추가선택(저스트냉모밀추가),리뷰이벤트입니다:）(양...",돈까스9
9,22**,"저스트등심돈카츠도시락/1,저스트카레우동/1",돈까스10


In [67]:
pork_cutlet_df.to_csv('./pork_cutlet.csv')

In [70]:
reviewer_list = []
menu_list = []
image_list = []
get_menu_list(url, addr, '죠스샌드위치', '샌드위치', reviewer_list, menu_list, image_list)

이미지 저장 ./샌드위치/샌드위치1.jpg
이미지 저장 ./샌드위치/샌드위치2.jpg
이미지 저장 ./샌드위치/샌드위치3.jpg
이미지 저장 ./샌드위치/샌드위치4.jpg
이미지 저장 ./샌드위치/샌드위치5.jpg
이미지 저장 ./샌드위치/샌드위치6.jpg
이미지 저장 ./샌드위치/샌드위치7.jpg
이미지 저장 ./샌드위치/샌드위치8.jpg
이미지 저장 ./샌드위치/샌드위치9.jpg
이미지 저장 ./샌드위치/샌드위치10.jpg
이미지 저장 ./샌드위치/샌드위치11.jpg
이미지 저장 ./샌드위치/샌드위치12.jpg
이미지 저장 ./샌드위치/샌드위치13.jpg
이미지 저장 ./샌드위치/샌드위치14.jpg
이미지 저장 ./샌드위치/샌드위치15.jpg
이미지 저장 ./샌드위치/샌드위치16.jpg
이미지 저장 ./샌드위치/샌드위치17.jpg
이미지 저장 ./샌드위치/샌드위치18.jpg
이미지 저장 ./샌드위치/샌드위치19.jpg
이미지 저장 ./샌드위치/샌드위치20.jpg
이미지 저장 ./샌드위치/샌드위치21.jpg
이미지 저장 ./샌드위치/샌드위치22.jpg
이미지 저장 ./샌드위치/샌드위치23.jpg
이미지 저장 ./샌드위치/샌드위치24.jpg
이미지 저장 ./샌드위치/샌드위치25.jpg
이미지 저장 ./샌드위치/샌드위치26.jpg
이미지 저장 ./샌드위치/샌드위치27.jpg
이미지 저장 ./샌드위치/샌드위치28.jpg
이미지 저장 ./샌드위치/샌드위치29.jpg
이미지 저장 ./샌드위치/샌드위치30.jpg
이미지 저장 ./샌드위치/샌드위치31.jpg
이미지 저장 ./샌드위치/샌드위치32.jpg
이미지 저장 ./샌드위치/샌드위치33.jpg
이미지 저장 ./샌드위치/샌드위치34.jpg
이미지 저장 ./샌드위치/샌드위치35.jpg
이미지 저장 ./샌드위치/샌드위치36.jpg
이미지 저장 ./샌드위치/샌드위치37.jpg
이미지 저장 ./샌드위치/샌드위치38.jpg
이미지 저장 ./샌드위치/샌드위치39.jpg
이미지 저장 ./샌드위치/샌드위치40.jpg
이미지 저장 ./

In [74]:
sandwich_df = pd.DataFrame({'reviewer':reviewer_list, 'menu':menu_list, 'image':image_list})
sandwich_df.head(10)

,reviewer,menu,image
0,2s**,"클럽샌드위치/1,숯불고기잡곡샌드위치/1,베이컨피자파니니/1",샌드위치1
1,jb**,"베이컨에그샌드위치/1,카야반반토스트/1",샌드위치2
2,po**,"파스트라미치아바타/1,웰치스포도/1",샌드위치3
3,im**,"클럽샌드위치/1,크림치즈베이글/1",샌드위치4
4,jo**,"클럽샌드위치/1,시저치킨샐러드/1",샌드위치5
5,jo**,"클럽샌드위치/1,시저치킨샐러드/1",샌드위치6
6,0r**,"햄치즈샌드위치/1,연어아보카도잡곡샌드위치/1,카야버터토스트/1",샌드위치7
7,yi**,"클럽샌드위치/1,햄치즈샌드위치/1",샌드위치8
8,yi**,"클럽샌드위치/1,햄치즈샌드위치/1",샌드위치9
9,jo**,"블루베리바나나쥬스/1,클럽샌드위치/1,연어아보카도잡곡샌드위치/1",샌드위치10


In [75]:
sandwich_df.to_csv('./sandwich.csv')

In [76]:
reviewer_list = []
menu_list = []
image_list = []
get_menu_list(url, addr, '김뿌라스시', '초밥', reviewer_list, menu_list, image_list)

이미지 저장 ./초밥/초밥1.jpg
이미지 저장 ./초밥/초밥2.jpg
이미지 저장 ./초밥/초밥3.jpg
이미지 저장 ./초밥/초밥4.jpg
이미지 저장 ./초밥/초밥5.jpg
이미지 저장 ./초밥/초밥6.jpg
이미지 저장 ./초밥/초밥7.jpg
이미지 저장 ./초밥/초밥8.jpg
이미지 저장 ./초밥/초밥9.jpg
이미지 저장 ./초밥/초밥10.jpg
이미지 저장 ./초밥/초밥11.jpg
이미지 저장 ./초밥/초밥12.jpg
이미지 저장 ./초밥/초밥13.jpg
이미지 저장 ./초밥/초밥14.jpg
이미지 저장 ./초밥/초밥15.jpg
이미지 저장 ./초밥/초밥16.jpg
이미지 저장 ./초밥/초밥17.jpg
이미지 저장 ./초밥/초밥18.jpg
이미지 저장 ./초밥/초밥19.jpg
이미지 저장 ./초밥/초밥20.jpg
이미지 저장 ./초밥/초밥21.jpg
이미지 저장 ./초밥/초밥22.jpg
이미지 저장 ./초밥/초밥23.jpg
이미지 저장 ./초밥/초밥24.jpg
이미지 저장 ./초밥/초밥25.jpg
이미지 저장 ./초밥/초밥26.jpg
이미지 저장 ./초밥/초밥27.jpg
이미지 저장 ./초밥/초밥28.jpg
이미지 저장 ./초밥/초밥29.jpg
이미지 저장 ./초밥/초밥30.jpg
이미지 저장 ./초밥/초밥31.jpg
이미지 저장 ./초밥/초밥32.jpg
이미지 저장 ./초밥/초밥33.jpg
이미지 저장 ./초밥/초밥34.jpg
이미지 저장 ./초밥/초밥35.jpg
이미지 저장 ./초밥/초밥36.jpg
이미지 저장 ./초밥/초밥37.jpg
이미지 저장 ./초밥/초밥38.jpg
이미지 저장 ./초밥/초밥39.jpg
이미지 저장 ./초밥/초밥40.jpg
이미지 저장 ./초밥/초밥41.jpg
이미지 저장 ./초밥/초밥42.jpg
이미지 저장 ./초밥/초밥43.jpg
이미지 저장 ./초밥/초밥44.jpg
이미지 저장 ./초밥/초밥45.jpg
이미지 저장 ./초밥/초밥46.jpg
이미지 저장 ./초밥/초밥47.jpg
이미지 저장 ./초밥/초밥48.jpg
이

In [77]:
sushi_df = pd.DataFrame({'reviewer':reviewer_list, 'menu':menu_list, 'image':image_list})
sushi_df.head(10)

,reviewer,menu,image
0,j0**,"착한초밥13p/1,돈까스덮밥（돈부리）/1",초밥1
1,j0**,"착한초밥13p/1,돈까스덮밥（돈부리）/1",초밥2
2,j0**,"착한초밥13p/1,돈까스덮밥（돈부리）/1",초밥3
3,dk**,"연어초밥홀릭12p/1,끝판왕초밥18p/1",초밥4
4,hb**,착한초밥13p/2,초밥5
5,st**,대박초밥15p/1,초밥6
6,st**,대박초밥15p/1,초밥7
7,st**,대박초밥15p/1,초밥8
8,14**,"도다리세꼬시회덮밥/1,메까갈릭불초밥1p/1,성게알초밥1p/1,연어알초밥1p/1",초밥9
9,hy**,"광어초밥1p/4,엔가와초밥1p/2,연어양파타르초밥1p/1,왕새우초밥1p/2,간장새우...",초밥10


In [78]:
sushi_df.to_csv('./sushi.csv')